# Apple Quality Prediction: An End-to-End Classification Project

## 1. Project Introduction

This notebook walks through the process of building a machine learning model to predict the quality of apples (`good` or `bad`) based on a set of physical and chemical features.

The project covers the following key stages:
1. **Data Loading and Cleaning:** Importing the dataset and handling missing or inconsistent data.
2. **Exploratory Data Analysis (EDA):** Visualizing the data to understand feature distributions and relationships.
3. **Data Preprocessing:** Preparing the data for machine learning models.
4. **Model Building and Evaluation:** Training several classification models and comparing their performance to select the best one.
5. **Feature Importance Analysis:** Identifying which characteristics are most influential in determining apple quality.

## 2. Data Loading and Cleaning

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
df = pd.read_csv('Apple_Quality_Dataset.csv')

In [ ]:
# Display the first 5 rows of the dataframe
df.head()

In [ ]:
# Check the data types and look for missing values
df.info()

**Observation:** The dataset has 4000 entries. There seems to be a single missing value in the `Acidity` column. We will drop any rows with missing data to ensure a clean dataset.

In [ ]:
# Drop rows with any missing values
df.dropna(inplace=True)

In [ ]:
# Verify that missing values have been handled
df.isnull().sum()

In [ ]:
# The 'A_id' column is just an identifier and provides no predictive value, so we drop it.
df.drop('A_id', axis=1, inplace=True)

In [ ]:
# Check for and remove any duplicate rows
print(f"Number of duplicate rows: {df.duplicated().sum()}")
df.drop_duplicates(inplace=True)
print(f"Number of rows after dropping duplicates: {len(df)}")

## 3. Exploratory Data Analysis (EDA)

### 3.1 Descriptive Statistics

In [ ]:
# Get a statistical summary of the numerical features
df.describe()

### 3.2 Target Variable Distribution

In [ ]:
# Check the distribution of the 'Quality' column
quality_counts = df['Quality'].value_counts()
print(quality_counts)

# Visualize the distribution
plt.figure(figsize=(6, 5))
sns.countplot(x='Quality', data=df, palette=['#FF6347', '#32CD32'])
plt.title('Distribution of Apple Quality')
plt.show()

**Observation:** The dataset is perfectly balanced, which is ideal for training a classification model as it avoids a natural bias towards one class.

### 3.3 Feature Distributions (Univariate Analysis)

In [ ]:
# Plot histograms for all numerical features
numerical_features = ['Size', 'Weight', 'Sweetness', 'Crunchiness', 'Juiciness', 'Ripeness', 'Acidity']
df[numerical_features].hist(bins=30, figsize=(15, 10), layout=(3, 3))
plt.suptitle("Histograms of Numerical Features")
plt.tight_layout(rect=[0, 0, 1, 0.96]) # Adjust layout to make space for suptitle
plt.show()

### 3.4 Feature Relationships (Bivariate Analysis)

In [ ]:
# Encode the 'Quality' for correlation analysis ('good': 1, 'bad': 0)
df['Quality_encoded_for_corr'] = df['Quality'].apply(lambda x: 1 if x == 'good' else 0)

# Create a correlation matrix
plt.figure(figsize=(10, 8))
corr_matrix = df[numerical_features + ['Quality_encoded_for_corr']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap of All Features')

# Save the figure to be used in the README
plt.savefig('images/correlation_heatmap.png', bbox_inches='tight')

plt.show()

# Drop the temporary encoded column
df.drop('Quality_encoded_for_corr', axis=1, inplace=True)

**Correlation Insights:**
- **Sweetness, Crunchiness, and Juiciness** show a moderate positive correlation with Quality.
- **Acidity** shows a notable negative correlation, meaning higher acidity is linked to lower quality.
- `Size` and `Weight` are highly correlated with each other, which is expected.

In [ ]:
# Boxplots to see feature distributions per quality category
for feature in numerical_features:
    plt.figure(figsize=(7, 5))
    sns.boxplot(x='Quality', y=feature, data=df, palette=['#FF6347', '#32CD32'])
    plt.title(f'{feature} Distribution by Apple Quality')
    plt.show()

## 4. Machine Learning Modeling

Now, we will prepare the data and train several classification models to predict apple quality.

### 4.1 Data Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Encode the target variable 'Quality' into numerical format
# 'good' will be 1, 'bad' will be 0
le = LabelEncoder()
df['Quality_encoded'] = le.fit_transform(df['Quality'])

# Define features (X) and target (y)
X = df[numerical_features]
y = df['Quality_encoded']

# Split the data into training (80%) and testing (20%) sets
# stratify=y ensures that the proportion of 'good' and 'bad' apples is the same in both train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

### 4.2 Training and Evaluating Models

We will train four different models and compare their performance.

In [ ]:
# Initialize the models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Support Vector Machine": SVC(random_state=42)
}

# Dictionary to store results
results = {}

# Loop through models, train, and evaluate
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    
    print(f"--- {name} ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred, target_names=['Bad', 'Good']))
    print("-"*30 + "\n")

### 4.3 Model Comparison

In [ ]:
# Create a DataFrame from the results dictionary
results_df = pd.DataFrame(list(results.items()), columns=['Model', 'Accuracy'])
results_df = results_df.sort_values(by='Accuracy', ascending=False)

print(results_df)

**Result:** The Random Forest Classifier performs the best with an accuracy of approximately 90.5%. Let's examine it more closely.

### 4.4 In-Depth Look at the Best Model: Random Forest

In [ ]:
# The best model is Random Forest
best_model = models["Random Forest"]
y_pred_best = best_model.predict(X_test)

# Plot the confusion matrix for the best model
cm = confusion_matrix(y_test, y_pred_best)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Predicted Bad', 'Predicted Good'], 
            yticklabels=['Actual Bad', 'Actual Good'])
plt.title('Confusion Matrix for Random Forest Classifier')
plt.show()

In [ ]:
# Feature Importance Analysis
importances = best_model.feature_importances_
feature_names = X.columns

# Create a DataFrame for visualization
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})

# Sort the features by importance
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# Plot the feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance_df, palette='viridis')
plt.title('Feature Importance for Apple Quality Prediction')
plt.show()

**Feature Importance Insight:** The model indicates that **Acidity**, **Ripeness**, and **Sweetness** are the top three most important features for predicting apple quality.

## 5. Conclusion

This project successfully built and evaluated several machine learning models to classify apple quality. 

**Key Findings:**
- **Best Model:** The **Random Forest Classifier** was the top-performing model, achieving an accuracy of **~90.5%** on the test set.
- **Important Features:** The analysis of feature importance revealed that `Acidity`, `Ripeness`, and `Sweetness` are the most significant predictors of an apple's quality.
- **Actionable Insight:** The strong negative correlation of `Acidity` and positive correlation of `Sweetness` confirm that these are primary drivers of consumer-perceived quality, providing a clear focus for quality control.

Overall, the project demonstrates a practical application of data science to solve a real-world classification problem, from initial data exploration to final model deployment and interpretation.